# Ref:
- https://github.com/TimS-ml/nanoGPT/blob/master/gpt.py
- https://github.com/lucidrains/x-transformers/blob/main/examples/enwik8_simple/train.py

wip
You can specify the data directory by setting the `DATA_DIR` environment variable.

In [1]:
import os
from boring_llm_base.constants import PROJECT_HOME_DIR
import sys; sys.path.append(str(PROJECT_HOME_DIR)); os.chdir(PROJECT_HOME_DIR)

In [2]:
import random
import tqdm
import gzip
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset

In [5]:
from boring_utils.utils import cprint

In [4]:
data_dir = os.getenv('DATA_DIR', './data')

# read nanoGPT's shakespeare_char
# with open(os.path.join(data_dir, 'shakespeare_char/input.txt'), 'r', encoding='utf-8') as f:
#     text = f.read()

# read enwik8 first 95M bytes
with gzip.open(os.path.join(data_dir, 'enwik8/enwik8.gz')) as file:
    text = file.read(int(95e6)).decode('utf-8')

# create mapping from characters to indices
chars = sorted(list(set(text)))
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }

# encode text
data = torch.tensor([stoi[c] for c in text], dtype=torch.long)
# n = int(0.9*len(data))
# data_train, data_val = data[:n], data[n:]
data_train, data_val = np.split(data, [int(90e6)])  # for enwik8

In [12]:
cprint(text[:1000])
cprint(text[10000:11000])
cprint(text[-1000:])
cprint(data_val[:300])

<module> -> text[:1000]:
('<mediawiki xmlns="http://www.mediawiki.org/xml/export-0.3/" '
 'xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" '
 'xsi:schemaLocation="http://www.mediawiki.org/xml/export-0.3/ '
 'http://www.mediawiki.org/xml/export-0.3.xsd" version="0.3" xml:lang="en">\n'
 '  <siteinfo>\n'
 '    <sitename>Wikipedia</sitename>\n'
 '    <base>http://en.wikipedia.org/wiki/Main_Page</base>\n'
 '    <generator>MediaWiki 1.6alpha</generator>\n'
 '    <case>first-letter</case>\n'
 '      <namespaces>\n'
 '      <namespace key="-2">Media</namespace>\n'
 '      <namespace key="-1">Special</namespace>\n'
 '      <namespace key="0" />\n'
 '      <namespace key="1">Talk</namespace>\n'
 '      <namespace key="2">User</namespace>\n'
 '      <namespace key="3">User talk</namespace>\n'
 '      <namespace key="4">Wikipedia</namespace>\n'
 '      <namespace key="5">Wikipedia talk</namespace>\n'
 '      <namespace key="6">Image</namespace>\n'
 '      <namespace key="7">Image talk</names

In [ ]:
class TextSamplerDataset(Dataset):
    def __init__(self, data, seq_len):
        super().__init__()
        self.data = data
        self.seq_len = seq_len

    def __getitem__(self, index):
        # randomly select start position
        rand_start = torch.randint(0, len(self.data) - self.seq_len - 1, (1,))
        full_seq = self.data[rand_start: rand_start + self.seq_len + 1].long()
        return full_seq.cuda()

    def __len__(self):
        return len(self.data) // self.seq_len

# modify decode function
def decode_tokens(tokens):
    return ''.join([itos[token.item()] for token in tokens])

In [ ]:
NUM_TOKENS = len(chars) # replace original 256
SEQ_LEN = 256  # keep consistent with nanoGPT